In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

mimic_demo_csv = pd.read_csv('data/mimic_demographics.csv', header=None, names=['id', 'year_of_birth', 'gender', 'race_long', 'race'])
mimic_demo_csv = mimic_demo_csv.replace("Black or African American", "Black")
mimic_demo_csv = mimic_demo_csv.replace("American Indian or Alaska Native", "AI/AN")
mimic_demo_csv = mimic_demo_csv.replace("Native Hawaiian or Other Pacific Islander", "NH/PI")
mimic_demo_csv = mimic_demo_csv.replace("No matching concept", "Unknown")
mimic_demo_csv = mimic_demo_csv.replace("Other Race", "Other")

conditions_csv = pd.read_csv('data/mimic_conditions.csv', header=None, names=['id', 'condition_id', 'condition_name'])
drugs_csv = pd.read_csv('data/mimic_drugs.csv', header=None, names=['id', 'drug_id', 'drug_name'])
drugs_csv = drugs_csv[drugs_csv.drug_name.str.contains("vaccine") == False]

conditions = mimic_demo_csv.merge(conditions_csv, how='inner', on='id')
drugs = mimic_demo_csv.merge(drugs_csv, how='inner', on='id')

mimic_demo_csv.head()

,id,year_of_birth,gender,race_long,race
0,186,2128,M,OTHER,Unknown
1,471,2076,M,UNKNOWN/NOT SPECIFIED,Unknown
2,475,2056,F,UNKNOWN/NOT SPECIFIED,Unknown
3,479,2121,M,UNKNOWN/NOT SPECIFIED,Unknown
4,487,2126,F,UNKNOWN/NOT SPECIFIED,Unknown


In [2]:
np.max(mimic_demo_csv['year_of_birth'])
np.min(mimic_demo_csv['year_of_birth'])

1800

### MIMIC Crosstabs on race and gender

* Age will have to use NLP if available (structured data is offset)

In [3]:
race_gender = pd.crosstab(index=mimic_demo_csv["race"], 
                            columns=[mimic_demo_csv["gender"]],
                            margins=True)
race_gender

gender,F,M,All
race,,,
AI/AN,21,26,47
Asian,744,947,1691
Black,2059,1816,3875
NH/PI,7,8,15
Other,50,59,109
Unknown,2813,4071,6884
White,14705,19194,33899
All,20399,26121,46520


### MIMIC Crosstabs on Conditions

In [4]:
conditions_ct = pd.crosstab(index=conditions["condition_name"], 
                            columns=[conditions["gender"],conditions["race"]],
                            margins=True)
conditions_ct = conditions_ct.sort_values('All', ascending=False)
conditions_ct.head(50)

gender                                                 F                     \
race                                               AI/AN Asian  Black NH/PI   
condition_name                                                                
All                                                  250  5337  31429   114   
Essential hypertension                                 3   178    975     4   
Congestive heart failure                               9    83    791     0   
Atrial fibrillation                                    3    96    402     2   
Coronary arteriosclerosis in native artery             1    57    373     1   
Type 2 diabetes mellitus                               6    92    637     2   
Acute renal failure syndrome                           5    84    609     0   
Hyperlipidemia                                         1    73    410     0   
Acute respiratory failure                              4    71    430     1   
Urinary tract infectious disease                       4    91    434     3   
Gastroesophageal reflux disease                        1    41    353     0   
Pure hypercholesterolemia                              3    50    266     2   
Sepsis                                                 3    78    325     0   
Hypertensive renal disease with renal failure          2    56    559     0   
Anemia                                                 3    59    381     1   
No matching concept                                    3    41    280     1   
Pneumonia                                              2    50    274     1   
Hypothyroidism                                         3    44    201     0   
Acidosis                                               4    59    334     0   
Acute posthemorrhagic anemia                           3    76    270     1   
Chronic obstructive lung disease                       1    11    143     0   
Pneumonitis due to inhalation of food or vomitus       0    34    141     1   
Depressive disorder                                    1    15    258     2   
Tobacco dependence syndrome                            0     7    152     3   
Acute subendocardial infarction                        1    22    109     0   
Old myocardial infarction                              3    18    113     1   
Hypovolemia                                            1    40    220     1   
Thrombocytopenic disorder                              0    35    180     0   
Hypo-osmolality and or hyponatremia                    4    41    174     1   
Pleural effusion                                       0    30    110     1   
Coronary bypass graft finding                          1    13     57     0   
Mitral valve disorder                                  0    27    145     1   
Hemorrhage AND/OR hematoma complicating procedure      0    28    121     2   
Septic shock                                           1    34    146     0   
Conduction disorder of the heart                       1    26    150     1   
Aortic valve disorder                                  2     5     54     0   
Cardiac complication                                   2    16     50     0   
Low blood pressure                                     1    25    140     0   
Acute tubular necrosis                                 1    22    130     0   
Premature - weight 1000g-2499g or gestation of ...     4    52    141     0   
Neonatal jaundice associated with preterm delivery     2    52    131     0   
Hyperosmolality and or hypernatremia                   0    15    174     1   
Atelectasis                                            0    25     86     0   
Hyperkalemia                                           4    17    179     0   
Drug-induced hypotension                               0    24     63     0   
Chronic pulmonary heart disease                        1    29    163     0   
Asthma                                                 1    17    244     1   
Trauma and postoperative pulmonary insufficiency  

### MIMIC Crosstabs on Drugs

In [5]:
drugs_ct = pd.crosstab(index=drugs["drug_name"], 
                            columns=[drugs["gender"],drugs["race"]],
                            margins=True)
drugs_ct = drugs_ct.sort_values('All', ascending=False)
drugs_ct.head(50)

gender                          F                                            \
race                        AI/AN  Asian  Black NH/PI Other Unknown   White   
drug_name                                                                     
All                           406  10232  54863   188   882   48824  342109   
Sodium Chloride                15    456   2155     7    37    1971   13648   
Glucose                        14    384   1877     7    33    1855   11911   
Potassium Chloride             10    354   1597     8    30    1705   11196   
Docusate                        7    277   1477     4    26    1071    8879   
heparin, porcine                8    287   1558     5    25    1166    8861   
Magnesium Sulfate              11    272   1317     5    22    1171    8865   
Acetaminophen                   8    213   1087     4    18    1118    7629   
pantoprazole                    8    220   1278     3    20    1058    7794   
Metoprolol                      6    181   1073     1    13     973    7225   
Furosemide                      9    170    907     3    12    1006    6862   
Oxycodone                       5    156    763     3    16     815    5386   
sennosides, USP                 6    188   1149     4    20     681    6144   
Vancomycin                      8    174    933     3    16     763    5546   
Propofol                        4    134    567     3     8     917    4792   
Aspirin                         5    109    705     3    12     809    4858   
Morphine                        3    188    752     4    15     655    5378   
Calcium Chloride                4    158    584     1    15     679    4978   
Lactate                         3    156    565     1    14     648    4839   
Bisacodyl                       3    137    728     2    17     568    4937   
Lorazepam                       6     89    648     3    10     615    4837   
Fentanyl                        7    153    654     3    10     699    4288   
Regular Insulin, Human          3     83    623     2     6     635    3351   
Ipratropium                     7     83    636     1     7     508    4031   
atorvastatin                    2     72    476     2     7     588    3333   
Ondansetron                     2    130    634     4    13     461    4292   
Chlorhexidine                   3     82    384     3     3     455    2814   
Albuterol                       7     68    536     0     6     484    3234   
Glucagon                        3    110    567     2    13     464    2819   
Hydralazine                     1     87    590     1     9     475    2977   
Warfarin                        3     57    387     1     4     376    2669   
Lidocaine                       2     53    394     1     9     265    2202   
Hydromorphone                   2     72    349     3    11     282    2429   
Dibasic potassium phosphate     2     86    357     3    10     328    2558   
Diphenhydramine                 1     61    398     1     9     355    2436   
Famotidine                      1     81    362     4     7     325    2294   
potassium phosphate             1     87    345     3     9     308    2443   
Midazolam                       4     79    346     3     3     377    2004   
Nitroglycerin                   0     45    247     0     3     271    1842   
Cefazolin                       0     67    169     1     5     251    1797   
Piperacillin                    2     73    339     1     4     270    1899   
tazobactam                      2     73    339     1     4     270    1899   
clopidogrel                     3     28    227     1     3     374    1804   
Metoclopramide                  0     38    192     1     4     243    1689   
Ampicillin                      3    143    320     0     6     338    1666   
Lactulose                       1     46    360     1     5     253    1696   
Sodium Bicarbonate              2     59    296     0     8     226    1680   
levothyroxine                   4     59    244   

### Comparing Vectors

In [6]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.wrappers.fasttext import FastText
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from collections import defaultdict
from prep_data import read_aact

In [7]:
data = read_aact()
df = pd.DataFrame(data)
df.columns = ['study_id', 'inclusion', 'exclusion', 'gender', 'age_min', 'age_max']
df.head()

6795


,study_id,inclusion,exclusion,gender,age_min,age_max
0,NCT00000105,Inclusion Criteria:\n - Patients mus...,Exclusion Criteria:\n - Pregnant or ...,Both,18 Years,N/A
1,NCT00000106,Inclusion Criteria:\n - Patients are...,,Both,18 Years,65 Years
2,NCT00000107,Inclusion Criteria:\n - Resting bloo...,,Both,17 Years,60 Years
3,NCT00000108,Inclusion Criteria:\n - Postmenopaus...,,Female,50 Years,65 Years
4,NCT00000102,Inclusion Criteria:\n - diagnosed wi...,Exclusion Criteria:\n - history of l...,Both,14 Years,35 Years


In [8]:
# cleanup and model building
df['inclusion'].replace('[\\n!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', inplace=True, regex=True)
inclusion = filter(None, [next_text.lower().strip(" ").split(" ") for next_text in df["inclusion"]])
model = Word2Vec(inclusion, size=100, window=5, min_count=5, workers=4)

In [9]:
model.wv.most_similar("hemophilia")

[('glomerulopathy', 0.40327781438827515),
 ('neither', 0.3382846415042877),
 ('minutes', 0.3372820019721985),
 ('pis', 0.3328104317188263),
 ('provider', 0.31975698471069336),
 ('percutaneous', 0.3175405263900757),
 ('myelotoxic', 0.30776888132095337),
 ('mouse', 0.30301523208618164),
 ('conforming', 0.3029557466506958),
 ('read', 0.2927960455417633)]

In [10]:
word_vectors = model.wv.vectors
n_words = word_vectors.shape[0]
vec_size = word_vectors.shape[1]
print("words = {0}, vector size = {1}".format(n_words, vec_size))

words = 5998, vector size = 100


In [11]:
cluster_size = 50
kmeans = KMeans(n_clusters=cluster_size, random_state=0).fit(word_vectors)
labels = kmeans.predict(word_vectors)
centers = kmeans.cluster_centers_

vocab_list = list(model.wv.vocab.keys())
word_indexes = dict()
for i in range(len(vocab_list)):
    word_indexes[i] = vocab_list[i]
vector_df = pd.DataFrame(word_vectors, index=vocab_list)

In [12]:
df_vocab = pd.DataFrame(vocab_list, columns=['word'])
df_vocab.insert((df_vocab.shape[1]),'kmeans', labels)
df_vocab.head()

,word,kmeans
0,,3
1,itself,38
2,neovascularization,28
3,pirfenidone,38
4,nephrectomy,38


In [13]:
for i in range(cluster_size):
    c = df_vocab[df_vocab['kmeans'] == i]
    print("cluster {0}, size {1}".format(i, c.shape[0]))
    print(list(c['word']))
    print()

cluster 0, size 128
['form', 'anthracyclines', 'pathological', 'resections', 'manifestation', 'main', 'described', 'pneumothorax', 'trastuzumab', 'rsr', 'bcc', 'uicc', 'reagents', 'trimethoprim', 'mesodermal', 'reflexes', 'review', 'exogenous', 'meperidine', 'sparing', 'refused', 'vein', 'megakaryocytes', 'hypertrophy', 'intraperitoneal', 'initial', 'neuromuscular', 'purified', 'distress', 'exploration', 'aspirin', 'hypo', 'interactions', 'lactating', 'tolerance', 'chairman', 'antihypertensives', 'sampled', 'deposition', 'sensitive', 'cc', 'spongioblastoma', 'glandular', 'astrocytoma', 'thoracotomy', 'examined', 'neutrophil', 'cortisol', 'etiology', 'partially', 'affected', 'filling', 'space', 'fluoropyrimidine', 'vestibular', 'valproic', 'longer', 'seronegativity', 'sulfadiazine', 'schwannoma', 'designate', 'tendon', 'implants', 'unusual', 'philadelphia', 'mr', 'aplasia', 'highest', 'ankle', 'pervasive', 'ic', 'their', 'lower', 'gondii', 'seminoma', 'promyelocytic', 'myeloid', 'autoan